In [22]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from PIL import Image
import gdown
import shutil
from tqdm import tqdm

In [23]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [24]:
df_train=pd.read_csv('/content/drive/MyDrive/ml семестр 6/проект/df_train.csv', sep=',')
df_test=pd.read_csv('/content/drive/MyDrive/ml семестр 6/проект/df_test.csv', sep=',')

In [25]:
df_train.head()

,index,weight,width,heigth,emotion,filename,hash,notsquare,lvalueld
0,45318,3059,209,209,sad,2285.jpg,1b99556874277005a9031de8fc7a82962625071ec992d0...,0,10.939264
1,21960,7878,379,379,fear,2803.jpg,9178043592bf422b34aaaa04a430681e854d9b682503d9...,0,11.865651
2,19812,3039,192,192,fear,4569.jpg,b07f20367ecfb0fabb6dc8ebd58e971c7c483c8ba16954...,0,12.455973
3,5815,15959,675,675,surprise,1152.jpg,2bd4b0d20cba4f5cadde1e6f0a962f69473d0a538407c4...,0,12.743730
4,20587,15547,616,616,fear,3409.jpg,0f3850caea2f6397ee3babb374441829b978825b13c4a8...,0,13.185115


In [26]:
df_test.head()

,index,weight,width,heigth,notsquare,image_path
0,4004.jpg,91362,1068,1068,0,4004.jpg
1,4003.jpg,8004,194,194,0,4003.jpg
2,4034.jpg,9322,204,204,0,4034.jpg
3,4033.jpg,12983,261,261,0,4033.jpg
4,4002.jpg,13682,304,304,0,4002.jpg


In [27]:
#маппинг эмоций
emotion_labels = {
    'anger': 0, 'contempt': 1, 'disgust': 2, 'fear': 3, 'happy': 4,
    'neutral': 5, 'sad': 6, 'surprise': 7, 'uncertain': 8
}

In [28]:
# обработка данных
class EmotionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = []
        self.labels = []

        # Заполнение списка файлов и меток
        for emotion, label in emotion_labels.items():
            emotion_dir = os.path.join(root_dir, emotion)
            for img_file in os.listdir(emotion_dir):
              # проверка на формат
                if img_file.endswith('.jpg'):
                    self.image_files.append(os.path.join(emotion_dir, img_file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    # возвращаем изображение и его метку
    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

# трансформируем изображения
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# создаем датасет и загрузчик
train_dataset = EmotionDataset(root_dir="/content/drive/MyDrive/ml семестр 6/проект/train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [29]:
# определение модели
class EmotionClassifier(nn.Module):
    def __init__(self):
        super(EmotionClassifier, self).__init__()
        self.model = models.mobilenet_v2(pretrained=True) # предобученная модель MobileNetV2
        self.model.classifier[1] = nn.Linear(self.model.last_channel, 9)

    def forward(self, x):
        return self.model(x)


In [30]:
model = EmotionClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# настройка устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


EmotionClassifier(
  (model): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 96, kernel_siz

In [31]:
num_epochs = 3
# обучение
for epoch in tqdm(range(num_epochs)):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

100%|██████████| 3/3 [3:52:58<00:00, 4659.64s/it]


In [42]:
#сохранение модели
torch.save(model.state_dict(), "emotion_classifier_model.pth")

In [34]:
# предсказание на тестовых данных
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.image_files[idx]

In [40]:
# датасет для тестирования
test_dataset = TestDataset(root_dir="/content/drive/MyDrive/ml семестр 6/проект/test_kaggle", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

In [41]:
# предсказание на тестовых данных
model.eval()
predictions = []
with torch.no_grad():
    for images, image_files in test_loader:
        images = images.to(device, non_blocking=True)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(zip(image_files, predicted.cpu().numpy()))

In [43]:
# обратный маппинг меток в эмоции
reverse_emotion_labels = {v: k for k, v in emotion_labels.items()}
predictions = [(img, reverse_emotion_labels[pred]) for img, pred in predictions]

In [44]:
submission_df = pd.DataFrame(predictions, columns=['image_path', 'emotion'])
submission_df.to_csv("/content/drive/MyDrive/ml семестр 6/проект/submission.csv", index=False)

In [45]:
print(submission_df.head())

  image_path    emotion
0   4004.jpg   contempt
1   4003.jpg        sad
2   4034.jpg  uncertain
3   4033.jpg   surprise
4   4002.jpg   contempt
